****Overview Of Code****

* This notebook focuses on predicting digitized ECG signals from the PhysioNet ECG Image Digitization competition.

* The pipeline starts by generating per-lead templates from the training dataset, where each lead signal is normalized and resampled to a fixed length to create a representative average waveform.

* These templates are then used to generate predictions for the test set by interpolating them to match the required number of rows per sample and applying smoothing filters to ensure realistic signal behavior.

* After normalization and scaling within the expected value range, the predictions for each sample and lead are stored in a structured format with id and value columns.

* The notebook also provides quality control (QC) metrics for each lead, including min, max, mean, and standard deviation of the predicted values to verify consistency.

* Example visualizations of test ECG signals are included to help understand the output signals.

* Finally, the predictions are saved as submission.csv, fully formatted for Kaggle submission, with the first few rows displayed as a table to ensure correctness.

* This pipeline ensures reproducibility, maintains signal fidelity, and produces submission-ready predictions efficiently.

In [ ]:
# PhysioNet ECG Digitization 

import os, warnings, random, time, gc
warnings.filterwarnings("ignore")
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from scipy.signal import butter, filtfilt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# ---------------- CONFIG ----------------
DATA_PATH = '/kaggle/input/physionet-ecg-image-digitization/'
WORK_DIR = '/kaggle/working'
os.makedirs(WORK_DIR, exist_ok=True)

TRAIN_CSV = os.path.join(DATA_PATH, 'train.csv')
TEST_CSV  = os.path.join(DATA_PATH, 'test.csv')
TRAIN_DIR = os.path.join(DATA_PATH, 'train')
SAMPLE_SUB = os.path.join(DATA_PATH, 'sample_submission.parquet')
SUBMISSION_CSV = os.path.join(WORK_DIR, 'submission.csv')

LEADS = ['I','II','III','aVR','aVL','aVF','V1','V2','V3','V4','V5','V6']
TEMPLATE_LEN = 500
MIN_VAL, MAX_VAL = 0.0, 0.07

SEED = 42
BASE_LR = 2e-4
EPOCHS = 35             # increase for final runs
BATCH_GPU = 64
BATCH_CPU = 16
PATIENCE = 6
ENSEMBLE = True         # Train 2 models with different seeds and average predictions
MODEL_VARIANTS = 2 if ENSEMBLE else 1

# Safe multiprocessing settings (use 0 to avoid cleanup errors)
NUM_WORKERS = 0
PIN_MEMORY = False  # will set later based on device

# ---------------- UTIL & SEED ----------------
def seed_all(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
PIN_MEMORY = True if device.type == 'cuda' else False
BATCH = BATCH_GPU if device.type == 'cuda' else BATCH_CPU

# ---------------- READ METADATA ----------------
train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)
sample_sub = pd.read_parquet(SAMPLE_SUB)
print(f"Train rows: {len(train_df)}, Test rows: {len(test_df)}")

# ---------------- SIGNAL FILTER ----------------
def bandpass_filter(sig, lowcut=0.5, highcut=40.0, fs=500, order=2):
    if sig is None:
        return np.zeros(TEMPLATE_LEN, dtype=np.float32)
    if len(sig) < 3:
        return np.interp(np.linspace(0,1,TEMPLATE_LEN), np.linspace(0,1,len(sig)), sig).astype(np.float32)
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    try:
        b, a = butter(order, [low, high], btype='band')
        out = filtfilt(b, a, sig)
    except Exception:
        out = sig
    out = np.interp(np.linspace(0,1,TEMPLATE_LEN), np.linspace(0,1,len(out)), out).astype(np.float32)
    return out

# ---------------- PRELOAD SIGNALS (RAM) ----------------
print("Preloading training signals to RAM (this speeds up training)...")
start = time.time()
unique_ids = train_df['id'].unique()
signal_cache = {}
for uid in tqdm(unique_ids, desc="Preloading train"):
    csv_path = os.path.join(TRAIN_DIR, str(uid), f"{uid}.csv")
    arrs = []
    if not os.path.exists(csv_path):
        arrs = [np.zeros(TEMPLATE_LEN, dtype=np.float32) for _ in LEADS]
    else:
        try:
            df = pd.read_csv(csv_path)
            for lead in LEADS:
                if lead in df.columns:
                    sig = df[lead].dropna().values.astype(np.float32)
                    sig = bandpass_filter(sig)
                    # per-lead standardization (per-record)
                    sig = (sig - sig.mean()) / (sig.std() + 1e-8)
                else:
                    sig = np.zeros(TEMPLATE_LEN, dtype=np.float32)
                arrs.append(sig)
        except Exception:
            arrs = [np.zeros(TEMPLATE_LEN, dtype=np.float32) for _ in LEADS]
    signal_cache[int(uid)] = np.stack(arrs, axis=0)
print("Done preloading train in", time.time()-start, "s")

print("Preloading test signals to RAM...")
start = time.time()
test_cache = {}
for uid in tqdm(test_df['id'].unique(), desc="Preloading test"):
    csv_path = os.path.join(TRAIN_DIR, str(uid), f"{uid}.csv")
    arrs = []
    if not os.path.exists(csv_path):
        arrs = [np.zeros(TEMPLATE_LEN, dtype=np.float32) for _ in LEADS]
    else:
        try:
            df = pd.read_csv(csv_path)
            for lead in LEADS:
                if lead in df.columns:
                    sig = df[lead].dropna().values.astype(np.float32)
                    sig = bandpass_filter(sig)
                    sig = (sig - sig.mean()) / (sig.std() + 1e-8)
                else:
                    sig = np.zeros(TEMPLATE_LEN, dtype=np.float32)
                arrs.append(sig)
        except Exception:
            arrs = [np.zeros(TEMPLATE_LEN, dtype=np.float32) for _ in LEADS]
    test_cache[int(uid)] = np.stack(arrs, axis=0)
print("Done preloading test in", time.time()-start, "s")

# ---------------- DATASET ----------------
class InMemoryECGDataset(Dataset):
    def __init__(self, meta_df, cache, leads, mode='train', augment=False):
        self.meta = meta_df.reset_index(drop=True)
        self.cache = cache
        self.leads = leads
        self.mode = mode
        self.augment = augment

    def __len__(self):
        return len(self.meta)

    def __getitem__(self, idx):
        row = self.meta.iloc[idx]
        uid = int(row['id'])
        arr = self.cache.get(uid, np.zeros((len(self.leads), TEMPLATE_LEN), dtype=np.float32)).copy()
        if self.mode == 'train' and self.augment:
            # small augmentations
            if np.random.rand() < 0.5:
                arr += np.random.normal(0, 0.01, arr.shape).astype(np.float32)
            if np.random.rand() < 0.25:
                scale = np.random.uniform(0.95, 1.05)
                arr *= scale
            if np.random.rand() < 0.2:
                shift = np.random.randint(-6, 7)
                arr = np.roll(arr, shift, axis=1)
        x = torch.from_numpy(arr).float()
        if self.mode == 'train':
            return x, x
        else:
            return x

# ---------------- MODEL (deeper Residual U-Net 1D) ----------------
class ResidualBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv1 = nn.Conv1d(in_ch, out_ch, 3, padding=1)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.act = nn.ReLU()
        self.conv2 = nn.Conv1d(out_ch, out_ch, 3, padding=1)
        self.bn2 = nn.BatchNorm1d(out_ch)
        self.skip = nn.Conv1d(in_ch, out_ch, 1) if in_ch != out_ch else nn.Identity()

    def forward(self, x):
        out = self.act(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        return self.act(out + self.skip(x))

class DeepUNet1D(nn.Module):
    def __init__(self, in_ch=12, base=32):
        super().__init__()
        self.enc1 = ResidualBlock(in_ch, base)
        self.enc2 = ResidualBlock(base, base*2)
        self.enc3 = ResidualBlock(base*2, base*4)
        self.pool = nn.MaxPool1d(2)
        self.up = nn.Upsample(scale_factor=2, mode='linear', align_corners=True)
        self.dec2 = ResidualBlock(base*4 + base*2, base*2)
        self.dec1 = ResidualBlock(base*2 + base, base)
        self.outc = nn.Conv1d(base, in_ch, 1)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        e1 = self.enc1(x)                      # (B, base, L)
        e2 = self.enc2(self.pool(e1))          # (B, base*2, L/2)
        e3 = self.enc3(self.pool(e2))          # (B, base*4, L/4)
        d2 = self.up(e3)
        d2 = torch.cat([d2, e2], dim=1)
        d2 = self.dec2(d2)
        d1 = self.up(d2)
        d1 = torch.cat([d1, e1], dim=1)
        d1 = self.dec1(d1)
        d1 = self.dropout(d1)
        return torch.tanh(self.outc(d1))

# ---------------- LOSS FUNCTIONS ----------------
def correlation_loss(pred, target, eps=1e-8):
    pred_flat = pred.view(pred.size(0), -1)
    targ_flat = target.view(target.size(0), -1)
    vx = pred_flat - pred_flat.mean(dim=1, keepdim=True)
    vy = targ_flat - targ_flat.mean(dim=1, keepdim=True)
    corr = (vx * vy).sum(dim=1) / (torch.sqrt((vx**2).sum(dim=1) * (vy**2).sum(dim=1)) + eps)
    return 1.0 - corr.mean()

# ---------------- TRAIN / VALIDATION PREP ----------------
unique_ids = train_df['id'].unique()
train_ids, val_ids = train_test_split(unique_ids, test_size=0.15, random_state=SEED)
train_meta = train_df[train_df['id'].isin(train_ids)].reset_index(drop=True)
val_meta = train_df[train_df['id'].isin(val_ids)].reset_index(drop=True)
print("Train meta size:", len(train_meta), "Val meta size:", len(val_meta))

# Create datasets
train_ds = InMemoryECGDataset(train_meta, signal_cache, LEADS, mode='train', augment=True)
val_ds   = InMemoryECGDataset(val_meta, signal_cache, LEADS, mode='train', augment=False)

train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
val_loader   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)

print("Batch:", BATCH, "num_workers:", NUM_WORKERS, "pin_memory:", PIN_MEMORY)

# ---------------- TRAINING & ENSEMBLE ----------------
def train_model(seed, model_id):
    # set seed per model for diversity
    seed_all(seed)
    model = DeepUNet1D(in_ch=len(LEADS), base=32).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=BASE_LR, weight_decay=1e-6)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.6, patience=3, verbose=False)
    l1 = nn.SmoothL1Loss()
    scaler = torch.cuda.amp.GradScaler(enabled=(device.type=='cuda'))

    best_val = 1e9
    wait = 0
    start = time.time()

    for epoch in range(1, EPOCHS+1):
        t0 = time.time()
        model.train()
        running = 0.0; it = 0
        for x,y in train_loader:
            x = x.to(device); y = y.to(device)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast(enabled=(device.type=='cuda')):
                pred = model(x)
                loss = l1(pred, y) + 0.35 * correlation_loss(pred, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            running += loss.item(); it += 1
        train_loss = running / max(1, it)

        # validation every 2 epochs to save time
        val_loss = None
        if epoch % 2 == 0 or epoch == EPOCHS:
            model.eval()
            vrun = 0.0; vit = 0
            with torch.no_grad():
                for x,y in val_loader:
                    x = x.to(device); y = y.to(device)
                    pred = model(x)
                    vloss = l1(pred, y) + 0.35 * correlation_loss(pred, y)
                    vrun += vloss.item(); vit += 1
            val_loss = vrun / max(1, vit)
            scheduler.step(val_loss)

        if val_loss is not None:
            print(f"[Model {model_id}] Epoch {epoch}/{EPOCHS} train_loss:{train_loss:.5f} val_loss:{val_loss:.5f} time:{time.time()-t0:.1f}s")
        else:
            print(f"[Model {model_id}] Epoch {epoch}/{EPOCHS} train_loss:{train_loss:.5f} (val skipped) time:{time.time()-t0:.1f}s")

        # checkpoint
        if val_loss is not None:
            if val_loss < best_val:
                best_val = val_loss; wait = 0
                ckpt = os.path.join(WORK_DIR, f"best_model_{model_id}.pth")
                torch.save(model.state_dict(), ckpt)
            else:
                wait += 1
                if wait >= PATIENCE:
                    print(f"[Model {model_id}] Early stopping.")
                    break

    total = time.time() - start
    print(f"[Model {model_id}] Finished training in {int(total)}s; best_val={best_val:.6f}")
    return os.path.join(WORK_DIR, f"best_model_{model_id}.pth")

# Train 1..MODEL_VARIANTS models
ckpt_paths = []
for i in range(MODEL_VARIANTS):
    seed = SEED + i + 1
    ckpt = train_model(seed, i+1)
    ckpt_paths.append(ckpt)
    # small cleanup
    gc.collect()
    torch.cuda.empty_cache()

# ---------------- BATCHED INFERENCE (safe) ----------------
print("Batched inference using saved checkpoints (ensemble if >1)...")
# Build test tensor array (num_test_ids, 12, TEMPLATE_LEN)
test_ids = list(test_df['id'].unique())
test_inputs = np.stack([test_cache.get(int(uid), np.zeros((len(LEADS), TEMPLATE_LEN), dtype=np.float32)) for uid in test_ids], axis=0)
test_tensor = torch.from_numpy(test_inputs).float()

# DataLoader with num_workers=0 (safe)
test_dataset = TensorDataset(test_tensor)
test_loader = DataLoader(test_dataset, batch_size=max(1, BATCH), shuffle=False, num_workers=0, pin_memory=PIN_MEMORY)

# accumulate predictions
ensemble_preds = np.zeros((len(test_ids), len(LEADS), TEMPLATE_LEN), dtype=np.float32)

for ckpt in ckpt_paths:
    model = DeepUNet1D(in_ch=len(LEADS), base=32).to(device)
    model.load_state_dict(torch.load(ckpt, map_location=device))
    model.eval()
    preds = []
    with torch.no_grad():
        for (batch_x,) in test_loader:
            batch_x = batch_x.to(device)
            out = model(batch_x).cpu().numpy()
            preds.append(out)
    preds = np.concatenate(preds, axis=0)  # (num_test_ids, 12, TEMPLATE_LEN)
    ensemble_preds += preds

ensemble_preds /= len(ckpt_paths)

# postprocess and map to submission dict
predictions = {}
for idx, uid in enumerate(test_ids):
    arr = ensemble_preds[idx]  # (12, TEMPLATE_LEN)
    for l_idx, lead in enumerate(LEADS):
        signal = arr[l_idx]
        mn, mx = signal.min(), signal.max()
        if mx - mn > 1e-8:
            s = (signal - mn) / (mx - mn)
        else:
            s = np.zeros_like(signal)
        s = MIN_VAL + s * (MAX_VAL - MIN_VAL)
        predictions[(int(uid), lead)] = s.astype(np.float32)

# ---------------- BUILD SUBMISSION ----------------
rows = []
for _, r in test_df.iterrows():
    uid = int(r['id']); lead = r['lead']; n = int(r['number_of_rows'])
    seq = predictions.get((uid, lead), np.full(n, (MIN_VAL+MAX_VAL)/2, dtype=np.float32))
    seq_resized = np.interp(np.linspace(0,1,n), np.linspace(0,1,TEMPLATE_LEN), seq)
    for i in range(n):
        rows.append({'id': f"{uid}_{i}_{lead}", 'value': float(seq_resized[i])})

submission_df = pd.DataFrame(rows)
submission_df.to_csv(SUBMISSION_CSV, index=False)
print("Saved submission:", SUBMISSION_CSV)

# ---------------- QUICK PLOT (one test sample) ----------------
try:
    sample_id = test_ids[0]
    fig, axs = plt.subplots(4,3, figsize=(12,8))
    for i, lead in enumerate(LEADS):
        ax = axs[i//3, i%3]
        ax.plot(predictions[(int(sample_id), lead)])
        ax.set_title(lead)
        ax.set_xticks([])
    plt.tight_layout()
    plt.show()
except Exception:
    pass

print("All done. You can now submit:", SUBMISSION_CSV)
